# Static Load Calculation

This code is used to calculate the static loads of the arm to find the holding torque required in each joint. The code outputs the Torque of DOF 2-5 in kg-cm, N-m and oz-in

Inputs: The Material Specification allows user to type in the Cross sectional area of the arm material to be used and the sproperties of said arm Material

The next section consists of the inputs for each arm segment where Degree of Freedom(DOF) 1 is located at the base and DOF 5 is the end effector

DOF 1 is not included in this code since it is not part of the arm so it plays no role on the holding torque of the arm.
DOF 2-4 have the same inputs of Arm,Motor,Igus Joint Mass and Length whereas DOF 5 has an additional input for the load to be carried

## Libraries

In [3]:
import pandas as pd
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

## Constants

In [21]:
g=9.81#gravity m/s^2
I=1 #Second Moment of Inertia
E=1 #Elastic Modulus

## Material Specifications

In [22]:
Area=19.19 #cm^2
rho=2.7 #material Density in g/cm^3
wg=(rho*Area)/1000 #Weight per unit length kg/cm

# Degree of Freedom Parameters

The first degree (base) is not included since it is rotary and does not contribute to torque calculations

Glossary:

L= Length of Arm in cm

IJ= Igus Joint Mass in kg

AW= Arm Mass in kg

MW= Motor Mass kg

CW= Combined Mass of Igus and Motor kg

WJ= Weight of joint N

AW= Arm Weight N

## 2nd Degree

In [23]:
IJ2=2.05
MW2=1.2
CW2=IJ2+MW2
WJ2=CW2*g

## 3rd Degree

In [24]:
L12=6#cm
L820=14.2#cm
L11=16.2#cm
IJ3=0.79
W12=.09#kg mass of section connecting 820 to RLD 30
Wv=.66#kg mass of Vacuum
W820=.2#kg mass of 820 section
W11=.39#kg mass of connection piece
MW3=1.3 #kg mass of motor and mount
CW3=IJ3+MW3
WJ3=CW3*g
AW12=W12*g
WV=Wv*g
AW820=W820*g
AW11=W11*g

## 4th Degree

In [25]:
L4=24.64
IJ4=0.41
AM4=0.38
MW4=1.3
CW4=IJ4+MW4
WJ4=CW4*g
AW4=AM4*g

## 5th Degree

In [26]:
L5=11.23#cm
IJ5=0#no Joint
AM5=.21#Arm Mass kg
MW5=1.2#Motor Weight
CW5=IJ5+MW5
WJ5=CW5*g#Weight N
AW5=AM5*g#Weight N
CM5=12.38#cm Center of Mass

## End Effector

In [27]:
Me=0.05#kg
Load=0#kg
We=Me*g#N
F=Load*g#N
mmt=0.054#mass of end effector mount
Wmt=mmt*g#N

## Weight

In [28]:
WT=F+Wmt+We+AW5+WJ5+WJ4+AW4+WJ3+AW11+WV+AW820+AW12+AW2+WJ2

In [29]:
print('Weight of Arm=',"%.2f" % WT,'N' )
MA=WT/g
print('Mass of Arm=',"%.2f" % MA,'kg' )

Weight of Arm= 116.13 N
Mass of Arm= 11.84 kg


## Center of Mass Distance

In [37]:
x1=9.34#cm
x2=7.37#cm
x3=19.75#cm
x4=12.4#cm
x5=10.41#cm
x6=5.45#cm
x7=25.91#cm
x8=9.38#cm
x9=33.22#cm
x10=25.18#cm
x11=20.87#cm
x12=18.09#cm
x13=4.33#cm
Lmax=L12+L11+L820+L4+L5+x1
print('Lmax=','%.2f'% Lmax,'cm')

Lmax= 81.61 cm


## Torque Calculations

In [31]:
T5=((We+F)*x1+Wmt*x2)/g
T4=((We+F)*x3+Wmt*x4+WJ5*x5+AW5*x6)/g
T3=((We+F)*(x7+x3)+Wmt*(x7+x4)+WJ5*(x7+x5)+AW5*(x7+x6)+WJ4*x7+AW4*x8)/g
T2=((We+F)*(x9+x7+x3)+Wmt*(x9+x7+x4)+WJ5*(x9+x7+x5)+AW5*(x9+x7+x6)+WJ4*(x9+x7)+AW4*(x9+x8)+(WJ3*x9)+AW12*x10+WV*x11+AW820*x12+AW11*x13)/g

## Print Results

In [32]:
T={ 'kg-cm': [round(T2,2),round(T3,2),round(T4,2),round(T5,2)],'N-m': [round(T2*.0981,2),round(T3*.0981,2),round(T4*.0981,2),round(T5*.0981,2)],'oz-in': [round(T2*13.89,2),round(T3*13.89,2),round(T4*13.89,2),round(T5*13.89,2)]}
pd.DataFrame(data=T,columns=['kg-cm','N-m','oz-in'], index=['DOF2', 'DOF3', 'DOF4', 'DOF5'])


,kg-cm,N-m,oz-in
DOF2,312.89,30.69,4346.09
DOF3,102.39,10.04,1422.22
DOF4,15.29,1.50,212.43
DOF5,0.86,0.08,12.01


## Igus Calculations

### RLD-50-S-48-ST

In [33]:
GR50S=48#Gear Ratio
MT50S=50#N m
Eff50S=0.35 #Efficiency

### RLD-30-S-50-ST

In [34]:
GR30S=50#Gear Ratio
MT30S=20#N m
Eff30S=0.35 #Efficiency

### RLD-20-S-38-ST

In [35]:
GR20S=38#Gear Ratio
MT20S=20#N m
Eff20S=0.40 #Efficiency

## Stepper Motors

### NEMA 23: 23HS22-2804D-E1000

In [28]:
x=sym.symbols ('x')#x in RPM
Motor23=0.0007*x+0.8765#N-m

### NEMA 34: 34HS31-5504D-E1000

In [31]:
y=sym.symbols('y')#y in  RPM
Motor34=3E-14*y**5 - 9E-11*y**4 + 9E-8*y**3 - 4E-5*y**2 - 0.0004*y + 3.0134 #N-m